In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/test.csv
/kaggle/input/house-prices-advanced-regression-techniques/train.csv


In [45]:
train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
train.head()


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [46]:
train.shape

(1460, 81)

In [47]:
train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [48]:
train['SalePrice'].describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

In [66]:
corr = train.corr()
df = pd.DataFrame(corr.abs().unstack().drop_duplicates(), columns = ['corr']).reset_index()
df_threshold = df[(df['corr']>=0.5)&(df['corr']<1)]
# print(df_threshold)
df_clean = df_threshold[(df_threshold.level_0 != 'SalePrice')&(df_threshold.level_1 != 'SalePrice')].sort_values('corr',ascending = False)
print(df_clean)

          level_0       level_1      corr
638    GarageCars    GarageArea  0.882475
226     YearBuilt   GarageYrBlt  0.825667
479     GrLivArea  TotRmsAbvGrd  0.825489
379   TotalBsmtSF      1stFlrSF  0.819530
429      2ndFlrSF     GrLivArea  0.687501
569  BedroomAbvGr  TotRmsAbvGrd  0.676620
305    BsmtFinSF1  BsmtFullBath  0.649212
256  YearRemodAdd   GarageYrBlt  0.642277
475     GrLivArea      FullBath  0.630012
436      2ndFlrSF  TotRmsAbvGrd  0.616423
433      2ndFlrSF      HalfBath  0.609707
164   OverallQual    GarageCars  0.600671
154   OverallQual     GrLivArea  0.593007
208     YearBuilt  YearRemodAdd  0.592855
626   GarageYrBlt    GarageCars  0.588920
144   OverallQual     YearBuilt  0.572323
406      1stFlrSF     GrLivArea  0.566024
627   GarageYrBlt    GarageArea  0.564567
165   OverallQual    GarageArea  0.562022
536      FullBath  TotRmsAbvGrd  0.554784
145   OverallQual  YearRemodAdd  0.550684
157   OverallQual      FullBath  0.550600
163   OverallQual   GarageYrBlt  0

In [51]:
corr.nlargest(20, 'SalePrice')

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
SalePrice,-0.021917,-0.084284,0.351799,0.263843,0.790982,-0.077856,0.522897,0.507101,0.477493,0.386420,...,0.324413,0.315856,-0.128578,0.044584,0.111447,0.092404,-0.021190,0.046432,-0.028923,1.000000
OverallQual,-0.028365,0.032628,0.251646,0.105806,1.000000,-0.091932,0.572323,0.550684,0.411876,0.239666,...,0.238923,0.308819,-0.113937,0.030371,0.064886,0.065166,-0.031406,0.070815,-0.027347,0.790982
GrLivArea,0.008273,0.074853,0.402797,0.263116,0.593007,-0.079686,0.199010,0.287389,0.390857,0.208171,...,0.247433,0.330224,0.009113,0.020643,0.101510,0.170205,-0.002416,0.050240,-0.036526,0.708624
GarageCars,0.016570,-0.040110,0.285691,0.154871,0.600671,-0.185758,0.537850,0.420622,0.364204,0.224054,...,0.226342,0.213569,-0.151434,0.035765,0.050494,0.020934,-0.043080,0.040522,-0.039117,0.640409
GarageArea,0.017634,-0.098672,0.344997,0.180403,0.562022,-0.151521,0.478954,0.371600,0.373066,0.296970,...,0.224666,0.241435,-0.121777,0.035087,0.051412,0.061047,-0.027400,0.027974,-0.027378,0.623431
TotalBsmtSF,-0.015415,-0.238518,0.392075,0.260833,0.537808,-0.171098,0.391452,0.291066,0.363936,0.522396,...,0.232019,0.247264,-0.095478,0.037384,0.084489,0.126053,-0.018479,0.013196,-0.014969,0.613581
1stFlrSF,0.010496,-0.251758,0.457181,0.299475,0.476224,-0.144203,0.281986,0.240379,0.344501,0.445863,...,0.235459,0.211671,-0.065292,0.056104,0.088758,0.131525,-0.021096,0.031372,-0.013604,0.605852
FullBath,0.005587,0.131608,0.198769,0.126031,0.550600,-0.194149,0.468271,0.439046,0.276833,0.058543,...,0.187703,0.259977,-0.115093,0.035353,-0.008106,0.049604,-0.014290,0.055872,-0.019669,0.560664
TotRmsAbvGrd,0.027239,0.040380,0.352096,0.190015,0.427452,-0.057583,0.095589,0.191740,0.280682,0.044316,...,0.165984,0.234192,0.004151,-0.006683,0.059383,0.083757,0.024763,0.036907,-0.034516,0.533723
YearBuilt,-0.012713,0.027850,0.123349,0.014228,0.572323,-0.375983,1.000000,0.592855,0.315707,0.249503,...,0.224880,0.188686,-0.387268,0.031355,-0.050364,0.004950,-0.034383,0.012398,-0.013618,0.522897


In [52]:
cols = corr.nlargest(10, 'SalePrice')['SalePrice'].index
print(cols)

Index(['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea',
       'TotalBsmtSF', '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt'],
      dtype='object')


In [53]:
import matplotlib.pyplot as plt
import seaborn as sns

# sns.set()
# sns.pairplot(train[cols], height=2.5)
# plt.show()

In [54]:
data_x = train[cols].values
print(data_x)

[[208500      7   1710 ...      2      8   2003]
 [181500      6   1262 ...      2      6   1976]
 [223500      7   1786 ...      2      6   2001]
 ...
 [266500      7   2340 ...      2      9   1941]
 [142125      5   1078 ...      1      5   1950]
 [147500      5   1256 ...      1      6   1965]]


In [55]:
data_y = train['SalePrice'].values
data_x = train[cols].values

length = int(len(data_y) * 0.6)

train_y = data_y[:length]
train_x = data_x[:length]
test_y = data_y[length:]
test_x = data_x[length:]

from sklearn.metrics import mean_squared_error
import numpy as np

In [56]:
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(train_x, train_y)
y_pred = lasso.predict(test_x)
np.sqrt(mean_squared_error(test_y, y_pred))

1.174188942602714e-05

In [57]:
from sklearn import linear_model

lr = linear_model.LinearRegression()
lr.fit(train_x, train_y)
y_pred = lr.predict(test_x)
np.sqrt(mean_squared_error(test_y, y_pred))

3.2792988904398566e-11

In [58]:
from sklearn import linear_model

reg = linear_model.Ridge(alpha = .5)
reg.fit(train_x, train_y)
y_pred = reg.predict(test_x)
np.sqrt(mean_squared_error(test_y, y_pred))

1.8597049646749486e-08

In [59]:
from sklearn import linear_model

bayesian = linear_model.BayesianRidge(5)
bayesian.fit(train_x, train_y)
y_pred = bayesian.predict(test_x)
np.sqrt(mean_squared_error(test_y, y_pred))

3.4584141388760217e-09